In [ ]:
%matplotlib inline

import numpy as np
import sys
sys.path.append('/usr/local/lib/python2.7/site-packages')
import cv2
import matplotlib.pyplot as plt
import time


def convertToRGB(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)



haar_smile_cascade = cv2.CascadeClassifier('/Users/Vicky/OneDrive/Projects/FaceAndSmileAnalyzer/classifiers/mouth.xml')
test = cv2.imread('/Users/Vicky/OneDrive/Projects/FaceAndSmileAnalyzer/test.jpg')

#cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
gray_img = test
gray = cv2.cvtColor(gray_img,cv2.COLOR_BGR2GRAY)

#Detect smile
smile = haar_smile_cascade.detectMultiScale(gray_img,
                                            scaleFactor= 1.7,
                                            minNeighbors=22,
                                            minSize=(25, 25));
print (len(smile))


plt.imshow(gray_img)
for (x,y,w,h) in smile:
    img2 = gray_img[y:y+h, x:x+w];
    cv2.rectangle(gray_img, (x,y), (x+w,y+h), (0,255,0), 2)


img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
#plt.imshow(img2)

corners = cv2.goodFeaturesToTrack(img2,25,0.01,10)
corners = np.int0(corners)

for i in corners:
    x,y = i.ravel()
    cv2.circle(img2,(x,y),3,255,-1)
    

plt.imshow(img2),plt.show()





In [22]:
# Face Matcher Using PCA

import os
import numpy as np
import sys
sys.path.append('/usr/local/lib/python2.7/site-packages')
import cv2
import matplotlib.pyplot as plt
import glob

#Step 1: Read the training set 
data_path = r'/Users/Vicky/OneDrive/Projects/FaceAndSmileAnalyzer/data/'
main_path = r'/Users/Vicky/OneDrive/Projects/FaceAndSmileAnalyzer/GrayScaleFaces/'
#faces = os.listdir(data_path) #Retrievs the list of images from the folder

image_space = []


faces = glob.glob("/Users/Vicky/OneDrive/Projects/FaceAndSmileAnalyzer/data/*.jpg")


for face in faces:
    img = cv2.imread(face)
    
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (200,200))
    
    cv2.imwrite(face, img)
    
    #convert img into np array
    arr = np.array(img)
    [w,h] = arr.shape[0], arr.shape[1]
    arr = arr.reshape(w*h)
    image_space.append(arr)

image_space = np.array(image_space)

mean_vector = np.mean(image_space, axis = 0)











In [23]:
cv2.imwrite("mean_img.jpg",mean_vector.reshape(200,200))


True

In [31]:
A = np.array(image_space)

for i in range(0,101):
    A[i,:] = abs(A[i,:] - mean_vector)
#A is the matrix of faces subtracted from mean image. 

In [32]:
import os
import numpy as np
import sys
sys.path.append('/usr/local/lib/python2.7/site-packages')
import cv2
import matplotlib.pyplot as plt


#Step 1: Read the training set 
data_path = "/Users/Vicky/OneDrive/Projects/FaceAndSmileAnalyzer/MeanSubdImages"




for i in range(0,101):
    temp = A[i,:]
    temp = temp.reshape(200,200)
    cv2.imwrite(data_path + "/" +str(i)+".jpg",temp)


In [40]:
B = A.T
c = np.matmul(A,B)  #L matrix
c.shape
#c is the scatter matrix

B.shape

(40000, 101)

In [82]:
#calculate eigen vectors and eigen values
from numpy import linalg as LA
from collections import OrderedDict

eigen_values, eigen_vectors = LA.eig(c)

eigen_dict = dict(zip(eigen_values, eigen_vectors))
#sorted_eigen_dict = collections.OrderedDict(sorted(eigen_dict.items()), key=lambda t:t[0],reverse=True)
sorted_eigen_dict = OrderedDict()

sorted_keys = sorted(eigen_dict.keys(),reverse = True)

for key in sorted_keys:
    if key > 300:
        sorted_eigen_dict[key] = eigen_dict[key]

print(sorted_eigen_dict.keys())







# for i in range(0,101):
#     temp = eigen_vectors[i,:]
#     #temp = temp.T
#     eigen_face = np.matmul(B,temp).reshape(200,200)
    
    




[12886.895056827707, 1414.3504893823822, 1326.7499423309825, 1289.2877089223014, 1242.0702573197702, 1226.4352856557318, 1183.5530102720454, 1105.509791912757, 1045.8092689871901, 1017.0640015720899, 981.36505901118619, 953.38199728590723, 933.88763434599878, 906.81016508424011, 883.77918342357623, 834.36787902119795, 825.95596006658775, 808.54232670862177, 777.89813931779292, 773.85483015171371, 715.02772539226623, 709.07148933777592, 671.99804736975318, 660.96535711670788, 631.12503344430388, 608.77426981762233, 560.08969559198817, 529.49958986486615, 526.19086433876748, 504.83686499970167, 486.87276351841774, 451.04248557733348, 444.36642948835998, 378.90010058514702, 377.45756106350206, 359.31561991269206, 353.46215165351833, 333.30037197623039, 306.6550187677251]
